# MaTREEd data preprocessing notebook
This notebook has been developed for preparing the data exposed by the MaTREEd web application (https://gisdevio.github.io/MaTREEd). More info about the project and the data preprocessing operations can be found at https://github.com/GISdevio/MaTREEd/blob/main/project-description.md.

0) Import necessary libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

1) Import raw data 

Original data links: 
- trees: https://challenge.greemta.eu/dataset/trees
- neighborhoods: https://challenge.greemta.eu/dataset/administrativeunits

In [ ]:
#create dataframes from raw data (trees and neighborhoods)
trees_csv = pd.read_csv(r'raw_data/trees_madrid.zip', compression='zip', sep=',', quotechar='"', error_bad_lines=False, encoding='utf8')
neigh = gpd.read_file(r'raw_data/neighborhoods_madrid.geojson')

# Make trees_csv a GeoDataframe
trees_geo = gpd.GeoDataFrame(trees_csv, geometry=gpd.points_from_xy(trees_csv['longitude'], trees_csv['latitude']), crs='EPSG:4326')

In [ ]:
# Check raw tree data
trees_csv.head()

In [ ]:
# Check raw neighborhoods data
neigh.head()

2) Preliminary operations on raw data

In [ ]:
# add CODBARRIO to trees_geo
trees_join = gpd.sjoin(trees_geo, neigh, how='left', op='within')

# Remove some columns to trees_geo and create the trees geodataframe for processing
trees = trees_join[['objectid',
 'specie_name',
 'senescence',
 'crown_diameter',
 'height',
 'trunk_girth',
 'geometry',
 'CODBARRIO']]

# Add area to neighbourhoods [Km2]
neigh['area_km2']= neigh['geometry'].to_crs({'init': 'epsg:32630'}).map(lambda p: p.area / 10**6)


3) Data Cleaning

In [ ]:
# Remove null values

# Remove trees with NaN values in columns senescence, crown_diameter, height and trunk_girth
df_nan = trees.dropna(subset = ['senescence', 'crown_diameter', 'height', 'trunk_girth'])

# Remove possible outliers (based on manual data exploration)
# Keep only remaining trees with values in column crown_diameter between the 1st and 99.99th percentile of the column
df_dc = df_nan.drop(df_nan[np.logical_or((df_nan.crown_diameter < df_nan.crown_diameter.quantile(0.01)),(df_nan.crown_diameter > df_nan.crown_diameter.quantile(0.9999)))].index)
# Keep only remaining trees with values in column height between the 1st and 99.999th percentile of the column 
df_a = df_dc.drop(df_dc[np.logical_or((df_dc.height < df_dc.height.quantile(0.01)),(df_dc.height > df_dc.height.quantile(0.99999)))].index)
# Keep only remaining trees with values in column trunk_girth between the 1st and 99.99th percentile of the column
df_p = df_a.drop(df_a[np.logical_or((df_a.trunk_girth < df_a.trunk_girth.quantile(0.01)),(df_a.trunk_girth > df_a.trunk_girth.quantile(0.9999)))].index)

# save the clean trees dataframe
trees_clean = df_p

# flush no more useful dataframes from memory
del[df_nan, df_dc, df_a, df_p, trees_join, trees_csv,trees_geo]


4) Compute trees statistics for neighbours

In [ ]:
# Add tree counts and percentages to neighborhoods (total and after cleaning)
count_tot = trees.CODBARRIO.value_counts().to_frame()
count_tot.rename(columns={'CODBARRIO':'count_tot'}, inplace=True)

count_clean = trees_clean.CODBARRIO.value_counts().to_frame()
count_clean.rename(columns={'CODBARRIO':'count_clean'}, inplace=True)

neigh['tree_tot']= neigh.set_index('CODBARRIO').join(count_tot).set_index(neigh.index)['count_tot']
neigh['tree_clean']= neigh.set_index('CODBARRIO').join(count_clean).set_index(neigh.index)['count_clean']

#  Add the percentage of complete tree records out of the total available (used to compute stats
neigh['tree_valid']= round((neigh['tree_clean'] / neigh['tree_tot'])*100, 2)

# Add perennifolio_count and caducifolio_count to neighbours (after cleaning)
trees_perennifolio = trees_clean[trees_clean["senescence"] == 'PERENNIFOLIO']
count_perennifolio = trees_perennifolio.CODBARRIO.value_counts().to_frame()
count_perennifolio.rename(columns={'CODBARRIO':'count_perennifolio'}, inplace=True)

trees_caducifolio = trees_clean[trees_clean["senescence"] == 'CADUCIFOLIO']
count_caducifolio = trees_caducifolio.CODBARRIO.value_counts().to_frame()
count_caducifolio.rename(columns={'CODBARRIO':'count_caducifolio'}, inplace=True)

neigh['count_perennifolio']= neigh.set_index('CODBARRIO').join(count_perennifolio).set_index(neigh.index)['count_perennifolio']
neigh['count_caducifolio']= neigh.set_index('CODBARRIO').join(count_caducifolio).set_index(neigh.index)['count_caducifolio']
neigh['perc_perennifolio']= round((neigh['count_perennifolio'] / neigh['tree_clean'])*100, 2)
neigh['perc_caducifolio']= round((neigh['count_caducifolio'] / neigh['tree_clean'])*100, 2)

# Add avg_trunk_girth, avg_crown_diameter, sum_diammetro_copa, avg_height, tree_cover to neighbours (after cleaning)
avg_crown_diameter=trees_clean.groupby('CODBARRIO')['crown_diameter'].mean().to_frame()
avg_crown_diameter.rename(columns={'crown_diameter':'avg_crown_diameter'}, inplace=True)

trees_clean['AREA_COPA']=pow(trees_clean['crown_diameter'],2)*np.pi/4
sum_area_copa=trees_clean.groupby('CODBARRIO')['AREA_COPA'].sum().to_frame()
sum_area_copa.rename(columns={'AREA_COPA':'sum_area_copa'}, inplace=True)

avg_height=trees_clean.groupby('CODBARRIO')['height'].mean().to_frame()
avg_height.rename(columns={'height':'avg_height'}, inplace=True)

avg_trunk_girth=trees_clean.groupby('CODBARRIO')['trunk_girth'].mean().to_frame()
avg_trunk_girth.rename(columns={'trunk_girth':'avg_trunk_girth'}, inplace=True)

neigh['avg_crown_diameter']= neigh.set_index('CODBARRIO').join(avg_crown_diameter).set_index(neigh.index)['avg_crown_diameter']
neigh['sum_area_copa']= neigh.set_index('CODBARRIO').join(sum_area_copa).set_index(neigh.index)['sum_area_copa']
neigh["tree_cover"]=round(neigh["sum_area_copa"]/(neigh["area_km2"]*10**6)*100,2)
neigh['avg_height']= neigh.set_index('CODBARRIO').join(avg_height).set_index(neigh.index)['avg_height']
neigh['avg_trunk_girth']= neigh.set_index('CODBARRIO').join(avg_trunk_girth).set_index(neigh.index)['avg_trunk_girth']


# Add name and percentage of the three most abundant species in each neighborhood
# Create a new DataFrame containing name and count of the three most abundant species for each neighborhood
data=[]
for index,row in neigh.iterrows():
    esp_trees = trees_clean[trees_clean["CODBARRIO"] == row.CODBARRIO]
    list_trees = esp_trees.specie_name.value_counts()[0:3].to_frame()
    list_trees.rename(columns={'specie_name':'count_especie'}, inplace=True)
    if len(list_trees)==1:
        data.append({'CODBARRIO':row.CODBARRIO,'especie_1':list_trees.index[0], 'especie_1_count':list_trees.count_especie[0] , 'especie_2':'', 'especie_2_count':None, 'especie_3':'', 'especie_3_count':None})
    elif len(list_trees)==2:
        data.append({'CODBARRIO':row.CODBARRIO,'especie_1':list_trees.index[0], 'especie_1_count':list_trees.count_especie[0] , 'especie_2':list_trees.index[1], 'especie_2_count':list_trees.count_especie[1], 'especie_3':'', 'especie_3_count':None})
    elif len(list_trees)==3:
        data.append({'CODBARRIO':row.CODBARRIO,'especie_1':list_trees.index[0], 'especie_1_count':list_trees.count_especie[0] , 'especie_2':list_trees.index[1], 'especie_2_count':list_trees.count_especie[1], 'especie_3':list_trees.index[2], 'especie_3_count':list_trees.count_especie[2] })
    else:
        data.append({'CODBARRIO':row.CODBARRIO,'especie_1':'', 'especie_1_count':None, 'especie_2':'', 'especie_2_count':None, 'especie_3':'', 'especie_3_count':None})

agg_list_tree = pd.DataFrame(data)


# Merge the new DataFrame and the neighbor DataFrame according to CODBARRIO column
neigh = pd.merge(neigh,agg_list_tree,on='CODBARRIO')
del agg_list_tree

# Compute the percentage of the species
neigh["especie_1_perc"]=round((neigh["especie_1_count"]/neigh['tree_clean'])*100,2)
neigh["especie_2_perc"]=round((neigh["especie_2_count"]/neigh['tree_clean'])*100,2)
neigh["especie_3_perc"]=round((neigh["especie_3_count"]/neigh['tree_clean'])*100,2)
neigh["other_perc"]=100-neigh["especie_1_perc"]-neigh["especie_2_perc"]-neigh["especie_3_perc"]

# flush no more useful dataframes from memory
del[avg_height, avg_crown_diameter, avg_trunk_girth, count_caducifolio, count_perennifolio, count_clean, count_tot,
    esp_trees, sum_area_copa, trees_caducifolio, trees_perennifolio]

5) Compute carbon dioxide stock and sequestration rate for trees and neighbours

In [ ]:
'''COMPUTE CARBON DIOXIDE INDICATORS'''

# function to return Carbon Dioxide stock [Kg CO2]
def Cstock(df):
    trees = df 
    c_stock_list = []
    for row in trees.iterfeatures():
        sen = row['properties']['senescence']
        d = (row['properties']['trunk_girth']/np.pi)*100
        if sen == "PERENNIFOLIO":
            c_stock = (0.16155*(d)**2.310647)*0.5*3.67
        else :
            c_stock = (0.035702*(d)**2.580671)*0.5*3.67
        c_stock_list.append(c_stock)
    return c_stock_list 

# function to return Carbon Dioxide sequestration rate [Kg CO2/year]
def Cseq(df): 
    trees = df
    c_seq_list = []
    for row in trees.iterfeatures():
        sen = row['properties']['senescence']
        c_stock = row['properties']["c_stock"]
        d = (row['properties']['trunk_girth']/np.pi)*100
        d_t1 = (-0.5425 + 0.3189*np.log(d)) + d 
        if sen == "PERENNIFOLIO":
            c_stock_t1 = (0.16155*(d_t1)**2.310647)*0.5*3.67
        else :
            c_stock_t1 = (0.035702*(d_t1)**2.580671)*0.5*3.67
        c_seq = c_stock_t1 - c_stock
        if c_seq < 0.0:
            c_seq = 0.0
            
        c_seq_list.append(c_seq)
    return c_seq_list 

# Apply Carbon functions
trees_clean["c_stock"] = Cstock(trees_clean) 
trees_clean["c_seq"] = Cseq(trees_clean) 

# Add cumulative Carbon Dioxide stock and Carbon Dioxide sequestration rate to neighborhoods
sum_c_stock=trees_clean.groupby('CODBARRIO')['c_stock'].sum().to_frame()
sum_c_stock.rename(columns={'c_stock':'sum_c_stock'}, inplace=True)

sum_c_seq=trees_clean.groupby('CODBARRIO')['c_seq'].sum().to_frame()
sum_c_seq.rename(columns={'c_seq':'sum_c_seq'}, inplace=True)

neigh['sum_c_stock']= neigh.set_index('CODBARRIO').join(sum_c_stock).set_index(neigh.index)['sum_c_stock']
neigh['sum_c_seq']= neigh.set_index('CODBARRIO').join(sum_c_seq).set_index(neigh.index)['sum_c_seq']

# normalize stats on neighborhood area
neigh['tree_tot_n'] = neigh['tree_tot']/neigh["area_km2"]
neigh['sum_c_stock_n'] = neigh['sum_c_stock']/neigh["area_km2"]
neigh['sum_c_seq_n'] = neigh['sum_c_seq']/neigh["area_km2"]

# Set NaN values of columns of interest to 0
cols=["tree_tot","avg_trunk_girth","avg_height","avg_crown_diameter","tree_cover","perc_perennifolio","perc_caducifolio","especie_1_perc","especie_2_perc","especie_3_perc","other_perc","tree_valid","sum_c_stock","sum_c_seq"]

for c in cols:
     neigh[c] = neigh[c].fillna(0)
     
# flush no more useful dataframes and variables from memory 
del[sum_c_seq, sum_c_stock, row, c, cols, data, index, list_trees]

In [ ]:
# Check final tree data
trees_clean.head()

In [ ]:
# Check final neighborhood data
neigh.head()

6) Export processed data

In [ ]:
#to save outputs 
#trees_clean.to_file('.../trees_madrid_clean.geojson', driver='GeoJSON')
#neigh.to_file('.../neighborhoods_madrid_stats.geojson', driver='GeoJSON')